<a href="https://colab.research.google.com/github/anish170805/ML-projects/blob/main/IPLpredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from sklearn import preprocessing

In [ ]:
ipl = pd.read_csv('/content/drive/MyDrive/ipl_data.csv')
ipl.head()

In [ ]:
ipl.info()

In [ ]:
data = ipl.copy()

In [ ]:
matches_per_venue = data[['mid','venue']].drop_duplicates()
matches_count = matches_per_venue.groupby('venue').size().sort_values(ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(x=matches_count.values, y= matches_count.index, hue=matches_count.index, palette='rainbow', legend=False)
plt.title('Number of Matches per Venue (using groupby)')
plt.xlabel('Number of Matches')
plt.ylabel('Venue')
plt.show()

In [ ]:
runs_by_batsman = data.groupby('batsman')['runs'].max().sort_values(ascending=False).head(10)
plt.figure(figsize=(12,6))
sns.barplot(x=runs_by_batsman.values, y=runs_by_batsman.index,palette="pastel")
plt.title('Top 10 Batsmen by Total Runs')
plt.xlabel('Total Runs')
plt.ylabel('Batsman')
plt.show()

In [ ]:
wickets_by_bowlers = data.groupby('bowler')['wickets'].max().sort_values(ascending=False).head(10)
plt.figure(figsize=(12,6))
sns.barplot(x=wickets_by_bowlers.values, y=wickets_by_bowlers.index,palette="pastel")
plt.title('Top 10 Bowlers by wickets taken')
plt.xlabel('Total wickets')
plt.ylabel('Bowler')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
data_encoded = data.copy()
cat_cols = data_encoded.select_dtypes('object').columns.drop('date')
label_encoders = {}
for col in cat_cols:
  le = LabelEncoder()
  data_encoded[col] = le.fit_transform(data_encoded[col])
  label_encoders[col] = le

In [ ]:
data_corr = data_encoded.drop(columns=['mid','date'], axis=1).corr()
plt.figure(figsize=(12,6))
sns.heatmap(data_corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
data_encoded.drop(columns=['wickets_last_5','runs_last_5'],inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
feature_cols = ['bat_team', 'bowl_team', 'venue', 'runs', 'wickets', 'overs', 'striker', 'batsman', 'bowler']
X = data_encoded[feature_cols]
y = data_encoded['total']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = keras.Sequential([keras.layers.Input(shape=(X_train_scaled.shape[1],)),
                          keras.layers.Dense(512,activation='relu'),
                          keras.layers.Dense(216,activation='relu'),
                          keras.layers.Dense(1, activation='linear')
                          ])
hubber_loss = keras.losses.Huber()
model.compile(optimizer='adam', loss=hubber_loss, metrics=['mae'])

In [ ]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=64, validation_data=(X_test_scaled, y_test))

In [ ]:
model_losses = pd.DataFrame(model.history.history)
model_losses.plot()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
predictions = model.predict(X_test_scaled)
mean_absolute_error(y_test, predictions)

In [ ]:
import numpy as np
import gradio as gr
import warnings
warnings.filterwarnings("ignore")

def predict_score_gradio(
    venue_name,
    batting_team_name,
    bowling_team_name,
    striker_name,
    bowler_name,
    current_runs,
    current_wickets,
    current_overs,
    striker_current_runs
):
    # Encode categorical features
    encoded_venue = label_encoders['venue'].transform([venue_name])[0]
    encoded_batting_team = label_encoders['bat_team'].transform([batting_team_name])[0]
    encoded_bowling_team = label_encoders['bowl_team'].transform([bowling_team_name])[0]
    encoded_striker = label_encoders['batsman'].transform([striker_name])[0] # This maps to 'batsman' feature
    encoded_bowler = label_encoders['bowler'].transform([bowler_name])[0]

    # Prepare input features in the correct order for the model
    # feature_cols = ['bat_team', 'bowl_team', 'venue', 'runs', 'wickets', 'overs', 'striker', 'batsman', 'bowler']
    input_features = [
        encoded_batting_team,
        encoded_bowling_team,
        encoded_venue,
        current_runs,
        current_wickets,
        current_overs,
        striker_current_runs, # This maps to 'striker' feature in feature_cols, representing striker's individual runs
        encoded_striker,      # This maps to 'batsman' feature in feature_cols, representing striker's ID
        encoded_bowler
    ]

    # Convert to numpy array and reshape
    input_array = np.array(input_features).reshape(1, -1)

    # Scale the input features using the pre-trained scaler
    input_array_scaled = scaler.transform(input_array)

    # Make prediction using the Keras model
    predicted_score = model.predict(input_array_scaled, verbose=0)[0][0]

    return f"Predicted Total Runs: {int(predicted_score)}"

# Create Gradio interface components
venue_input = gr.Dropdown(list(label_encoders['venue'].classes_), label="Select Venue")
batting_team_input = gr.Dropdown(list(label_encoders['bat_team'].classes_), label="Select Batting Team")
bowling_team_input = gr.Dropdown(list(label_encoders['bowl_team'].classes_), label="Select Bowling Team")
striker_input = gr.Dropdown(list(label_encoders['batsman'].classes_), label="Select Striker")
bowler_input = gr.Dropdown(list(label_encoders['bowler'].classes_), label="Select Bowler")

runs_input = gr.Number(value=0, label="Current Runs")
wickets_input = gr.Number(value=0, label="Current Wickets")
overs_input = gr.Number(value=0.0, label="Current Overs")
striker_current_runs_input = gr.Number(value=0, label="Striker's Current Runs")

# Launch the Gradio interface
iface = gr.Interface(
    fn=predict_score_gradio,
    inputs=[
        venue_input,
        batting_team_input,
        bowling_team_input,
        striker_input,
        bowler_input,
        runs_input,
        wickets_input,
        overs_input,
        striker_current_runs_input
    ],
    outputs="text",
    title="IPL Score Predictor",
    description="Enter the match details to predict the total score."
)

iface.launch(debug=False, share=True)